In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-dataframe-for-yelp/user_df
/kaggle/input/user-dataframe-for-yelp/__results__.html
/kaggle/input/user-dataframe-for-yelp/__resultx__.html
/kaggle/input/user-dataframe-for-yelp/__notebook__.ipynb
/kaggle/input/user-dataframe-for-yelp/__output__.json
/kaggle/input/user-dataframe-for-yelp/custom.css
/kaggle/input/clean-review-hemani/clean_review.pkl


In [149]:
# Importing necessary libraries 
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import seaborn as sns
sns.set()
import json 
import os
import csv
import json
import sqlite3
from pathlib import Path
from typing import Dict, Text
from collections import defaultdict
%matplotlib inline
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

In [24]:
# Importing our preprocessed review table
cleaned_review = pd.read_pickle("../input/clean-review-hemani/clean_review.pkl")

#### The cleaned_review table contains only the business_ids of restraunts or places which serve food.</br>We have also droped all the columns except review_id, user_id, business_id and stars.

In [25]:
# Having a quick look at the cleaned review
cleaned_review.head()

,review_id,user_id,business_id,stars
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4
1,hpcZLEzqD4_gPi6eSVi_Bg,Y-j2svl0M_5-jF1ehYuNPQ,buF9druCkbuXLX526sGELQ,2
2,3FvY1Se8y2WXqTbaANOqMw,xUCX4GhBpeWxZB0l2lmt_w,buF9druCkbuXLX526sGELQ,5
3,C1uQNP2ehBktS43ZRMEvkg,2M6KFsWIUXElqcQRz4A0Qg,buF9druCkbuXLX526sGELQ,5
4,Cja8_35_kQDnF9g4voikzw,t5SRIRU6INiAyVkiMJhRPA,buF9druCkbuXLX526sGELQ,1


In [26]:
# create a dataframe grouped_by_userid which groups by the user_id and stores business_id, user_id and stars
grouped_by_userid = pd.DataFrame(cleaned_review.groupby('user_id')['business_id','user_id','stars'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [27]:
# having a quick look at the shape of the grouped_by_userid table
grouped_by_userid.shape

(1617955, 2)

#### As we can see from the shape above there are 1617955 unique users

In [28]:
# having a quick look at a dataframe for one random user
grouped_by_userid[1][1066660]

,review_id,user_id,business_id,stars
658784,hvWVO9z7tl75by8bqUeUXA,eCBEtYYF2lJmgwF8f8Nbrw,zioLxtBc9THNS2TOn9xW1w,1
3456223,xio4MPvQQGSAbAl9N2sY8A,eCBEtYYF2lJmgwF8f8Nbrw,dowDBpVQVsbkBjEpVq62Zg,5
5857585,aHR_6LtexhiGaoOi3iP4NA,eCBEtYYF2lJmgwF8f8Nbrw,6GmUVd6ZD7L8weD5dV1g3Q,5


#### We tried to load the whole dataset in such a manner that the indexes consist of the business_ids and the columns consist of user_ids. This caused our RAM to crash multiple times.</br></br> We then decided to take a small sample of the data by picking the first 100 users and all the businesses in the review column. This caused increased sparsity, as there were many businesses which were not even rated once and were in the table.</br></br>Finally we decided to pick the first 1000 users which have given more than 100 reviews and created a table with only those users.</br>

In [29]:
# Creating a dictionary of all the users who have given more than 100 reviews. 
# We are doing this as an attempt to decrease sparsity
# Creating an empty dictionary
s = {}
# looping through all the users
for i in range(1617955):
    # creating a temporary variable user_1 which stores all the reviews given by a given user
    user_1 = grouped_by_userid[1][i]
    # creating a list of all review indexes for that particular user
    idx = user_1.index
    # if the user has given more than 100 reviews we add the no of reviews and the user index to the s dictionary
    if len(idx)>100:
        s[i] = len(idx)

In [30]:
# creating a list which contains the userid and indexed according to the grouped_by_userid
index_userid = list(grouped_by_userid[0])

In [31]:
index_userid.index('-0eV_VL0_e0asdPv4u4EjQ')

666

In [41]:
# creating a list of all the users who have given reviews more than 100
idx= list(s.keys())

In [42]:
# Writing a function which adds all of a ratings for all the businesses rated by a particular user
def create1col(i):
    user_1 = grouped_by_userid[1][i]
    idx = user_1.index
    x = pd.DataFrame(index = user_1['business_id'].unique(), columns=[user_1['user_id'][idx[0]]])
    for i in idx:
        x[user_1['user_id'][idx[0]]][user_1['business_id'][i]] = user_1['stars'][i]
    return x

In [43]:
# concatenating the first 2 users review tables using outer join
x = pd.concat([create1col(idx[0]),create1col(idx[1])], axis=1)

In [45]:
# concatenating the dataframes for users which have the highest ratings
for i in idx[2:1000]:
    x = pd.concat([x,create1col(i)], axis=1)

In [46]:
# creating a function which adds a particular user in the dataframe
def adduser(x,userid):
    i = index_userid.index(userid)
    if i not in idx:
        x = pd.concat([x,create1col(i)], axis=1)
    else:
        print("user already in matrix")
    return x

In [142]:
#Adding the user for whom we want to generate the recommendation
itembased = adduser(x,'-5uvOHRkvijt1vzSX9RqzQ')

user already in matrix


In [151]:
# Finding the sparsity of the matrix
sparsity = (itembased.isna().values.sum()/itembased.size)*100
print("The sparsity of the matrix is :" + str(round(sparsity,2)) + "%")

The sparsity of the matrix is :99.58%


#### We will now center the values around 0. We do this to handle sparsity of the matrix. We are doing this because we want to fill all the nan values with 0. If we don't center all our values around 0

In [152]:
# Centering the values around 0
# Get the average rating for each user
avg_ratings = itembased.T.mean(axis=1)

# Center each user's ratings around 0
item_ratings_table_centered = itembased.T.sub(avg_ratings, axis=0)

# Fill in the missing data with 0s
item_ratings_table_normed = item_ratings_table_centered.fillna(0)

In [153]:
# Decompose the matrix 
U, sigma, Vt = svds(item_ratings_table_normed)
# create a diagonal matrix with the sigma
sigma = np.diag(sigma)

In [154]:
# Dot product of U and sigma
U_sigma = np.dot(U, sigma)

In [155]:
# Dot product of result and Vt
U_sigma_Vt = np.dot(U_sigma, Vt)

# Add back on the row means contained in avg_ratings
uncentered_ratings = U_sigma_Vt + avg_ratings.values.reshape(-1, 1)

# Create DataFrame of the results
calc_pred_ratings_df = pd.DataFrame(uncentered_ratings, 
                                    index=itembased.T.index,
                                    columns=itembased.T.columns
                                   )

In [156]:
# creating a function to recognize null values
def isNaN(num):
    return num!= num

In [157]:
# Extract the actual values
actual_values = itembased.T.values

# Extract the predicted values
predicted_values = calc_pred_ratings_df.values

# Create a mask of actual_values to only look at the non-missing values in the ground truth
mask = ~isNaN(actual_values)

# Print the mean squared error between the actual values and the predicted values
print(mean_squared_error(actual_values[mask], predicted_values[mask]))

0.926576512089588


In [190]:
# importing itemname_itemid 
itemname_itemid = pd.read_pickle('../input/itemname-itemid/hemani.pkl')

In [192]:
# creating a function called recommendforuser which gives us top 5 suggestions for a user
def recommendforuser(userid):
    m = calc_pred_ratings_df.loc[userid].sort_values(ascending=False)[:5]
    item_1_ratings = pd.DataFrame(m).reset_index().rename(columns={'index':'business_id'})
    item_1_ratings = item_1_ratings.merge(itemname_itemid,on='business_id',how='inner')
    return item_1_ratings

In [194]:
# calling the recommendforuser function for a particualr user
recommendforuser('-5uvOHRkvijt1vzSX9RqzQ')

,business_id,-5uvOHRkvijt1vzSX9RqzQ,name
0,I4jytmnWi2m9qGnuqNxUTg,3.333858,Sockeye City Grill
1,243F-wnRuvt2ehf4RUDoeQ,3.333686,King Buffet
2,TQj4HSCmm5GnriKMiSpqpQ,3.333663,Raisu
3,n1iC0j8DGAOfrkhQqxmSfA,3.333501,McDonald's
4,J3a6sldLoUpGlbco6JOmig,3.333466,California Hong Kong Cafe
